In [ ]:

!pip install setuptools
!pip install packaging
!pip install pandas dash



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.0 MB/s eta 0:00:00


In [ ]:
from dash import jupyter_dash

In [ ]:
jupyter_dash.default_mode='external'

In [ ]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"


In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                               options=[
                                                {'label': 'All Sites', 'value': 'All'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                ],
                                                placeholder='Select A Launch Site Here',
                                                value='ALL',
                                                style={'width': '50%'}
                                                ),


                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=1000,
                                                step=50, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'All':
        fig = px.pie(spacex_df, names='Launch Site', title='Total success launches for all sites')
    else:
      filtered_df=spacex_df[spacex_df['Launch Site'] == entered_site]
      filtered_df=filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
      fig = px.pie(filtered_df, values='class count', names='class', title=f"Total Success Launches for site {entered_site}")
    return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def scatter(entered_site, payload):
    filtered_dfs = spacex_df[spacex_df['Payload Mass (kg)'].between(min_payload, max_payload)]

    if entered_site == 'ALL':
      fig = px.scatter(filtered_dfs, x='Payload Mass (kg)', y='class', lebels='Booster Version Category', title='Success count on Payload mass for all sites')
    else:
        fig = px.scatter(filtered_dfs[filtered_dfs['Launch Site'] == entered_site], x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f"Success count on Payload mass for site {entered_site}")
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

<ipython-input-5-a5068b5537f4>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-5-a5068b5537f4>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
app.run(jupyter_server_url='SpaceX launch data ')

Dash app running on:


<IPython.core.display.Javascript object>